<a href="https://colab.research.google.com/github/jdetras/prepare4imagebreed/blob/main/prepare_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import functions and mount Google Drive**

In [ ]:
import glob
import re 
import os
import shutil
import os.path
import re
import pandas as pd
import zipfile
import yaml
import pprint

from os import path
from os.path import exists
from google.colab import drive
from google.colab import files
from zipfile import ZipFile
from yaml.loader import SafeLoader

drive.mount('/content/gdrive/', force_remount=True)

**Import yaml file containing details of the trial**

Upload a prepare_images.yaml file into Google Colab

In [ ]:
def main():
    with open('/content/prepare_images.yaml') as f:
        config = yaml.safe_load(f)
    return config
read_yaml = main()
pprint.pprint(read_yaml)

In [43]:
#define variables and directories from yaml file

#imaging event
imaging_event = (read_yaml['imaging_event'])

#directories
input_path = (read_yaml['input_path']).rstrip('/')
trial_path = (read_yaml['trial_path']).rstrip('/')
output_path = (read_yaml['output_path']).rstrip('/')
path = input_path + "/" + imaging_event + "/" + imaging_event #path for the input 

#excel file variable
field_trial_name = (read_yaml['field_trial_name']) #must match field trial defined in Imagebreed
geojson_filename = (read_yaml['geojson_filename']) #compressed geojson file
geojson_file = (read_yaml['geojson_file']) #uncompressed geojson file
date_time = (read_yaml['date_time']) #data format

#tif files to copy
dsm_path = (read_yaml['dsm_path'])
reflectance_path = (read_yaml['reflectance_path'])

#template directory. choose between re and rgb accordingly
data = pd.read_csv((read_yaml['template_excel']))

In [ ]:
#Check directories
print(sorted(os.listdir(input_path)))
# Connect to that directory:
os.chdir(input_path)

**Get list of images (tif files)**

In [ ]:
images = []
dsm = [f for f in glob.glob(path + "/" + dsm_path, recursive=True)]
reflectance = [f for f in glob.glob(path + "/" + reflectance_path, recursive=True)]

for tif in dsm:
    images.append(tif)

for tif in reflectance:
    images.append(tif)

print(images)

In [19]:
#create directory for the imaging event
output_path_imaging_event = output_path + "/" + imaging_event

if os.path.exists(output_path_imaging_event) == False:
    os.mkdir(output_path_imaging_event)

In [20]:
#copy images from source to output destination
for tif in images:
    shutil.copy(tif, output_path_imaging_event)

In [21]:
#regex pattern to for the image files
rename_images = os.listdir(output_path_imaging_event)

dsm_in = re.compile(r'[a-z0-9]_dsm.tif')
rgb_in = re.compile(r'[a-z0-9]_group1.tif')
blue_in = re.compile(r'[a-z0-9]_blue.tif')
green_in = re.compile(r'[a-z0-9]_green.tif')
nir_in = re.compile(r'[a-z0-9]_nir.tif')
rededge_in = re.compile(r'[a-z0-9]_red edge.tif')
red_in = re.compile(r'[a-z0-9]_red.tif')

In [22]:
#modify the tif files to Imagebreed filename standards
for file in rename_images:
    if dsm_in.search(file):
        dsm_out = file.replace("_dsm.tif","_dsm__bw.tif")
        if exists(output_path_imaging_event + "/" + dsm_out):
            print(dsm_out + " exists")
        else:  
            os.rename(output_path_imaging_event + "/" + file, \
            output_path_imaging_event + "/" + dsm_out)
    elif rgb_in.search(file):
        rgb_out = file.replace("group1.tif","group1__rgb.tif")
        if exists(output_path_imaging_event + "/" + rgb_out):
            print(rgb_out + " exists")
        else:
            os.rename(output_path_imaging_event + "/" + file, \
            output_path_imaging_event + "/" + rgb_out)
    elif blue_in.search(file):
        blue_out = file.replace("_blue.tif","__blue.tif")
        if exists(output_path_imaging_event + "/" + blue_out):
            print(blue_out + " exists")
        else:
            os.rename(output_path_imaging_event + "/" + file, \
            output_path_imaging_event + "/" + blue_out)
    elif green_in.search(file):
        green_out = file.replace("_green.tif","__green.tif")
        if exists(output_path_imaging_event + "/" + green_out):
            print(green_out + " exists")
        else:
            os.rename(output_path_imaging_event + "/" + file, \
            output_path_imaging_event + "/" + green_out)
    elif nir_in.search(file):
        nir_out = file.replace("_nir.tif","__nir.tif")
        if exists(output_path_imaging_event + "/" + nir_out):
            print(nir_out + " exists")
        else:
            os.rename(output_path_imaging_event + "/" + file, \
            output_path_imaging_event + "/" + nir_out)
    elif rededge_in.search(file):
        rededge_out = file.replace("_red edge.tif","__rededge.tif")
        if exists(output_path_imaging_event + "/" + rededge_out):
            print(rededge_out + " exists")
        else:
            os.rename(output_path_imaging_event + "/" + file, \
            output_path_imaging_event + "/" + rededge_out)
    elif red_in.search(file):
        red_out = file.replace("_red.tif","__red.tif")
        if exists(output_path_imaging_event + "/" + red_out):
            print(red_out + " exists")
        else:
            os.rename(output_path_imaging_event + "/" + file, \
            output_path_imaging_event + "/" + red_out)

In [23]:
#create list of images for the imaging event excel file
image_files = (sorted(f for f in os.listdir(output_path_imaging_event) \
                      if not f.startswith('.')))

images = ''
for f in image_files:
  images = images + f + ","
images = images.rstrip(images[-1])
#print(images)

In [25]:
os.chdir(output_path)
# writing files to a zipfile
with ZipFile(imaging_event + '.zip','w') as zip:
    # writing each file one by one
    for file in image_files:
        os.chdir(output_path_imaging_event)
        zip.write(file)

**Create an excel file for the imaging event requirement by Imagebreed**

In [ ]:
#add info for the excel file
data.iloc[[0],[0]] = imaging_event
data.iloc[[0],[3]] = 'Drone images for ' + imaging_event
data.iloc[[0],[4]] = date_time
data.iloc[[0],[8]] = field_trial_name
data.iloc[[0],[9]] = geojson_filename
data.iloc[[0],[10]] = images

print(data)

imaging_event_output = output_path + "/" + imaging_event + '_imagingevent.xls'
data.to_excel(imaging_event_output,index=False)

**Download Imagingevent excel file, orthophotomosaic zipped file and geojson file**

The geojson file will be download only once per trial.
If orthophotomosaic zip file fails to download due to disconnection of Google Colab, download directly from Google Drive. 

In [36]:
files.download(imaging_event_output)
files.download(output_path + "/" + imaging_event + ".zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#get geojson file and compress
if os.path.exists(output_path + "/" + geojson_filename) == False:
    shutil.copy(trial_path + "/" + geojson_file, output_path)
    os.chdir(output_path)
    with ZipFile(geojson_file + '.zip', 'w') as zip:
        zip.write(geojson_file)
    files.download(geojson_filename)
else:
    print('geojson_filename already exists')

**Delete temporary files**



1.   Uncompressed geojson file
2.   Uncompressed orthophotomosaic folder with images



In [ ]:
#delete geojson 
if os.path.exists(output_path + "/" + geojson_file):
    os.remove(output_path + "/" + geojson_file)
    print(geojson_file + ' deleted')
else:
    print('geojson_file does not exists')

In [ ]:
#delete uncompressed imaging event directory
if os.path.exists(output_path + "/" + imaging_event):
    shutil.rmtree(output_path + "/" + imaging_event)
    print(output_path + "/" + imaging_event + ' deleted')
else:
    print(output_path + "/" + imaging_event + ' does not exists')